In [1]:
import json
import os
import re

from openai import OpenAI
from dotenv import load_dotenv
from IPython.display import Markdown

In [ ]:
load_dotenv()

In [9]:
client = OpenAI(
    api_key=os.getenv("OPENAI_API_KEY"),
)

model_name = "gpt-3.5-turbo"

In [10]:
def self_feedback(user_prompt: str, max_iterations: int = 3, target_rating: int = 90):
    system_prompt = """An input is provided in <<>>. Evaluate the input in terms of how well it addresses the original task of explaining the key events and significance of the American Civil War. Consider factors such as: Breadth and depth of context provided; Coverage of major events; Analysis of short-term and long-term impacts/consequences; If you identify any gaps or areas that need further elaboration: Return output as json {"rating": "scoring in percentage", "feedback": "detailed feedback","revised_output": "return an improved output describing the key events and significance of the American Civil War. Avoid special characters like apostrophes (') and double quotes (\")"}. """
    revised_output = "The response is sufficient."
    feedback = ""
    for i in range(max_iterations):
        prompt = (
            user_prompt
            if i == 0
            else f"Feedback: {feedback}, Revised Output: {revised_output}"
        )
        messages = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": f"{prompt}"},
        ]
        response = client.chat.completions.create(model=model_name, messages=messages)
        content = response.choices[0].message.content

        # convert to valid json
        content_json = json.loads(content)

        # extract rating
        rating_str = content_json["rating"]
        rating_num = int(re.findall(r"\d+", rating_str)[0])

        # extract feedback
        feedback = content_json["feedback"]

        # extract revised output
        revised_output = content_json["revised_output"]
        print(
            f"i={i}, Rating: {rating_num}, \nFeedback: {feedback}, \nRevised Output: {revised_output}\n\n"
        )
        if rating_num >= target_rating:
            return revised_output

    return revised_output

In [11]:
user_prompt = "The American Civil War was a civil war in the United States between north and south."
res = self_feedback(user_prompt=user_prompt, max_iterations=5, target_rating=95)

i=0, Rating: 20, 
Feedback: The input provides a very basic and general overview of the American Civil War as a conflict between the North and the South. However, it lacks depth, context, and detail about key events, causes, and impacts of the war. It is crucial to cover major battles, key figures, causes of the war, such as slavery and states' rights, as well as the consequences like the Emancipation Proclamation and the end of slavery., 
Revised Output: The American Civil War was a major armed conflict fought from 1861 to 1865 between the Union (Northern states) and the Confederacy (Southern states) over issues primarily related to slavery and states' rights. Key events like the Battle of Gettysburg, the Emancipation Proclamation issued by President Lincoln, the Assassination of Abraham Lincoln, and the surrender of General Lee at Appomattox Court House were pivotal during this period. The war resulted in the abolition of slavery, the preservation of the Union, and a redefinition of 

In [14]:
Markdown(res)

The American Civil War, lasting from 1861 to 1865, was a pivotal conflict between the Union states in the North and the Confederacy states in the South. The primary causes were rooted in disagreements over slavery, states' rights, and economic differences. Notable battles like Gettysburg, Antietam, and Vicksburg played significant roles in shaping the outcome of the war. Key figures such as Abraham Lincoln, Jefferson Davis, Ulysses S. Grant, and Robert E. Lee influenced its course. The Emancipation Proclamation, issued by Lincoln in 1863, and the surrender of General Lee to General Grant at Appomattox Court House were critical events. The assassination of President Lincoln in 1865 added a tragic dimension to its conclusion. The war's long-term impacts included the abolition of slavery through the 13th Amendment, the reunification of the nation, and a shift towards increased federal power over states' rights, reshaping the country's political landscape for years to come.